Data automation tool 

Aim - to have a folder of radial velocity data in 2 different formats and READY to be inputted into the EXOFAST tool. Currently its in a very unpractical format and location.

The problem: Currently we have the data in urls. We first Manually select one at a time each url, open it, webscrape the data and save into csv files.

The solution: I want to build a python script to automatically do EVERYTHING.

Stage 1: Read the  separate urls off of the txt file into a array or dictionary

Stage 2: Webscrape each of the selected URLS and save the contents into a dataframe in pandas.
 
Stage 3: In each of the dataframes put the data into the 1 or 2 ideal formats that we want and save these correctly formatted data frames into txt files and csv files we want.


In [41]:
import re 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import csv
import datetime
import os


In [25]:
file_name = 'exoplanetarchive.ipac.caltech.edu_bulk_data_download_wget_RADIAL.bat.txt'

with open(file_name, 'r') as file:
    contents = file.read()
    
#print(contents) 
#Ignore this cell it is just for debugging purposes

Below is a function that reads a text file, searches for URLs using a regular expression pattern and returns a list of extracted URLs. 

In [39]:
def extract_urls_from_file():
    urls = []
    url_pattern = r'http://exoplanetarchive\.ipac\.caltech\.edu:80/data/ExoData/\d+/\d+/data/UID_\d+_RVC_\d+\.tbl'
    file_path = 'exoplanetarchive.ipac.caltech.edu_bulk_data_download_wget_RADIAL.bat.txt'
    
    with open(file_path, 'r') as file:
        contents = file.read()
        matches = re.findall(url_pattern, contents)
        urls = matches
        
    return urls

urls = extract_urls_from_file()
print(urls[9])


http://exoplanetarchive.ipac.caltech.edu:80/data/ExoData/0001/0001640/data/UID_0001640_RVC_002.tbl


In [32]:
# Iterate over each URL
for index, url in enumerate(urls):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the data from the webpage
        data = soup.get_text()

        # Split the data into rows
        rows = data.split("\n")

        # Ignore the first 19 rows (Metadata)
        relevant_rows = rows[22:]

        # Create a DataFrame from the relevant data
        df = pd.DataFrame([row.split() for row in relevant_rows])

        # Ensure that the DataFrame has three columns
        if df.shape[1] != 3:
            raise ValueError("Invalid number of columns in the DataFrame.")

        # Rename the columns
        df.columns = ['JD double days', 'RV double m/s', 'RV uncertainty double m/s']

        # Convert the columns to numeric types if needed
        df['JD double days'] = pd.to_numeric(df['JD double days'], errors='coerce')
        df['RV double m/s'] = pd.to_numeric(df['RV double m/s'], errors='coerce')
        df['RV uncertainty double m/s'] = pd.to_numeric(df['RV uncertainty double m/s'], errors='coerce')

        # Define a function to convert Julian dates
        def julianDateV2(jdate1):
            if pd.isnull(jdate1):
                return jdate1

            baseDate = datetime.datetime(2000, 1, 1)
            days = int(jdate1) - 2451545  # Subtract the Julian date of J2000 epoch

            # Add the fractional part of the Julian date as seconds
            seconds = round((jdate1 % 1) * 24 * 60 * 60)
            delta = datetime.timedelta(days=days, seconds=seconds)

            datestd = baseDate + delta
            return datestd.date().strftime('%Y-%m-%d')

        # Apply the conversion function to the 'JD double days' column
        df['JD double days'] = df['JD double days'].apply(julianDateV2)

        # Drop rows with NaN values in 'JD double days' column
        df = df.dropna(subset=['JD double days'])

        # Generate the output file name based on the index
        output_file = f"data{index}.csv"

        # Save the formatted data to the output CSV file
        df.to_csv(output_file, index=False)

        print(f"Data for URL {index} saved successfully.")
    else:
        print(f"Failed to retrieve data from URL {index}")

Data for URL 0 saved successfully.
Data for URL 1 saved successfully.
Data for URL 2 saved successfully.
Data for URL 3 saved successfully.
Data for URL 4 saved successfully.
Data for URL 5 saved successfully.
Data for URL 6 saved successfully.
Data for URL 7 saved successfully.


ValueError: Invalid number of columns in the DataFrame.

The below script scrapes data from a webpage and processes the extracted data to save into a CSV file.

First it sends a GET request to a specified URL and retrieves the HTML content of the webpage. The Beautiful soup library is then used to parse the HTML and extract the relevant data. The extracted data is split into rows and stored in a Pandas DataFrame. 

Next, the DataFrame is cleaned and formatted. The columns are renamed to more meaningful names and any neccasary datatype conversions are peformed, In this case the JD double days column is converted to regular Gregorian calender dates using a custom conversion function,

To handle potential NaN values in the JD double days column, the code checks for NaN values before applying the conversion function. Rows with NaN values are dropped from the DataFrame. 

Finally the formatted data is saved to a CSV file using the 'to_csv()" method.

Overally the code combines webscraping, data preprocessing and CSV file handling to automate the extraction and formatting of data from a webpage

In [37]:
# URL of the webpage containing the data
url = urls[8]

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the data from the webpage
    data = soup.get_text()

    # Split the data into rows
    rows = data.split("\n")

    # Ignore the first 19 rows (Metadata)
    relevant_rows = rows[22:]

    # Create a DataFrame from the relevant data
    df = pd.DataFrame([row.split() for row in relevant_rows])

    # Ensure that the DataFrame has three columns
    if df.shape[1] != 3:
        raise ValueError("Invalid number of columns in the DataFrame.")

    # Rename the columns
    df.columns = ['JD double days', 'RV double m/s', 'RV uncertainty double m/s']

    # Convert the columns to numeric types if needed
    df['JD double days'] = pd.to_numeric(df['JD double days'], errors='coerce')
    df['RV double m/s'] = pd.to_numeric(df['RV double m/s'], errors='coerce')
    df['RV uncertainty double m/s'] = pd.to_numeric(df['RV uncertainty double m/s'], errors='coerce')

    # Define a function to convert Julian dates
    def julianDateV2(jdate1):
        if pd.isnull(jdate1):
            return jdate1

        baseDate = datetime.datetime(2000, 1, 1)
        days = int(jdate1) - 2451545  # Subtract the Julian date of J2000 epoch

        # Add the fractional part of the Julian date as seconds
        seconds = round((jdate1 % 1) * 24 * 60 * 60)
        delta = datetime.timedelta(days=days, seconds=seconds)

        datestd = baseDate + delta
        return datestd.date().strftime('%Y-%m-%d')

    # Apply the conversion function to the 'JD double days' column
    df['JD double days'] = df['JD double days'].apply(julianDateV2)

    # Drop rows with NaN values in 'JD double days' column
    df = df.dropna(subset=['JD double days'])

    # Save the formatted data to a new CSV file
    df.to_csv('data.csv', index=False)

    print("Data saved successfully.")
else:
    print("Failed to retrieve data from the webpage")


ValueError: Invalid number of columns in the DataFrame.

In [36]:
import numpy as np

for index, url in enumerate(urls):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the data from the webpage
        data = soup.get_text()

        # Split the data into rows
        rows = data.split("\n")

        # Ignore the first 19 rows (Metadata)
        relevant_rows = rows[22:]

        # Create a DataFrame from the relevant data
        df = pd.DataFrame([row.split() for row in relevant_rows])

        # Ensure that the DataFrame has three columns
        if df.shape[1] != 3:
            raise ValueError("Invalid number of columns in the DataFrame.")

        # Rename the columns
        df.columns = ['JD double days', 'RV double m/s', 'RV uncertainty double m/s']

        # Convert the columns to numeric types with NaN values
        df['JD double days'] = pd.to_numeric(df['JD double days'], errors='coerce')
        df['RV double m/s'] = pd.to_numeric(df['RV double m/s'], errors='coerce')
        df['RV uncertainty double m/s'] = pd.to_numeric(df['RV uncertainty double m/s'], errors='coerce')

        # Generate the output file name based on the index
        output_file = f"data{index}.csv"

        # Save the formatted data to the output CSV file
        df.to_csv(output_file, index=False, na_rep='NaN')

        print(f"Data for URL {index} saved successfully.")
    else:
        print(f"Failed to retrieve data from URL {index}")


Data for URL 0 saved successfully.
Data for URL 1 saved successfully.
Data for URL 2 saved successfully.
Data for URL 3 saved successfully.
Data for URL 4 saved successfully.
Data for URL 5 saved successfully.
Data for URL 6 saved successfully.
Data for URL 7 saved successfully.


ValueError: Invalid number of columns in the DataFrame.

In [40]:
import numpy as np

for index, url in enumerate(urls):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the data from the webpage
        data = soup.get_text()

        # Split the data into rows
        rows = data.split("\n")

        # Ignore the first 19 rows (Metadata)
        relevant_rows = rows[22:]

        # Create a DataFrame from the relevant data
        df = pd.DataFrame([row.split() for row in relevant_rows])

        # Convert the columns to numeric types with NaN values
        df = df.apply(pd.to_numeric, errors='coerce')

        # Generate the output file name based on the index
        output_file = f"data{index}.csv"

        # Save the formatted data to the output CSV file
        df.to_csv(output_file, index=False, na_rep='NaN')

        print(f"Data for URL {index} saved successfully.")
    else:
        print(f"Failed to retrieve data from URL {index}")


Data for URL 0 saved successfully.
Data for URL 1 saved successfully.
Data for URL 2 saved successfully.
Data for URL 3 saved successfully.
Data for URL 4 saved successfully.
Data for URL 5 saved successfully.
Data for URL 6 saved successfully.
Data for URL 7 saved successfully.
Data for URL 8 saved successfully.
Data for URL 9 saved successfully.
Data for URL 10 saved successfully.
Data for URL 11 saved successfully.
Data for URL 12 saved successfully.
Data for URL 13 saved successfully.
Data for URL 14 saved successfully.
Data for URL 15 saved successfully.
Data for URL 16 saved successfully.
Data for URL 17 saved successfully.
Data for URL 18 saved successfully.
Data for URL 19 saved successfully.
Data for URL 20 saved successfully.
Data for URL 21 saved successfully.
Data for URL 22 saved successfully.
Data for URL 23 saved successfully.
Data for URL 24 saved successfully.
Data for URL 25 saved successfully.
Data for URL 26 saved successfully.
Data for URL 27 saved successfully.
Da

KeyboardInterrupt: 

In [42]:
# Create the "rawData" directory if it doesn't exist
os.makedirs("rawData", exist_ok=True)

for index, url in enumerate(urls):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the data from the webpage
        data = soup.get_text()

        # Split the data into rows
        rows = data.split("\n")

        # Ignore the first 19 rows (Metadata)
        relevant_rows = rows[22:]

        # Create a DataFrame from the relevant data
        df = pd.DataFrame([row.split() for row in relevant_rows])

        # Convert the columns to numeric types with NaN values
        df = df.apply(pd.to_numeric, errors='coerce')

        # Generate the output file path based on the index
        output_file = f"rawData/data{index}.csv"

        # Save the formatted data to the output CSV file
        df.to_csv(output_file, index=False, na_rep='NaN')

        print(f"Data for URL {index} saved successfully.")
    else:
        print(f"Failed to retrieve data from URL {index}")


Data for URL 0 saved successfully.
Data for URL 1 saved successfully.
Data for URL 2 saved successfully.
Data for URL 3 saved successfully.
Data for URL 4 saved successfully.
Data for URL 5 saved successfully.
Data for URL 6 saved successfully.
Data for URL 7 saved successfully.
Data for URL 8 saved successfully.
Data for URL 9 saved successfully.
Data for URL 10 saved successfully.
Data for URL 11 saved successfully.
Data for URL 12 saved successfully.
Data for URL 13 saved successfully.
Data for URL 14 saved successfully.
Data for URL 15 saved successfully.
Data for URL 16 saved successfully.
Data for URL 17 saved successfully.
Data for URL 18 saved successfully.
Data for URL 19 saved successfully.
Data for URL 20 saved successfully.
Data for URL 21 saved successfully.
Data for URL 22 saved successfully.
Data for URL 23 saved successfully.
Data for URL 24 saved successfully.
Data for URL 25 saved successfully.
Data for URL 26 saved successfully.
Data for URL 27 saved successfully.
Da

Data for URL 225 saved successfully.
Data for URL 226 saved successfully.
Data for URL 227 saved successfully.
Data for URL 228 saved successfully.
Data for URL 229 saved successfully.
Data for URL 230 saved successfully.
Data for URL 231 saved successfully.
Data for URL 232 saved successfully.
Data for URL 233 saved successfully.
Data for URL 234 saved successfully.
Data for URL 235 saved successfully.
Data for URL 236 saved successfully.
Data for URL 237 saved successfully.
Data for URL 238 saved successfully.
Data for URL 239 saved successfully.
Data for URL 240 saved successfully.
Data for URL 241 saved successfully.
Data for URL 242 saved successfully.
Data for URL 243 saved successfully.
Data for URL 244 saved successfully.
Data for URL 245 saved successfully.
Data for URL 246 saved successfully.
Data for URL 247 saved successfully.
Data for URL 248 saved successfully.
Data for URL 249 saved successfully.
Data for URL 250 saved successfully.
Data for URL 251 saved successfully.
D

Data for URL 447 saved successfully.
Data for URL 448 saved successfully.
Data for URL 449 saved successfully.
Data for URL 450 saved successfully.
Data for URL 451 saved successfully.
Data for URL 452 saved successfully.
Data for URL 453 saved successfully.
Data for URL 454 saved successfully.
Data for URL 455 saved successfully.
Data for URL 456 saved successfully.
Data for URL 457 saved successfully.
Data for URL 458 saved successfully.
Data for URL 459 saved successfully.
Data for URL 460 saved successfully.
Data for URL 461 saved successfully.
Data for URL 462 saved successfully.
Data for URL 463 saved successfully.
Data for URL 464 saved successfully.
Data for URL 465 saved successfully.
Data for URL 466 saved successfully.
Data for URL 467 saved successfully.
Data for URL 468 saved successfully.
Data for URL 469 saved successfully.
Data for URL 470 saved successfully.
Data for URL 471 saved successfully.
Data for URL 472 saved successfully.
Data for URL 473 saved successfully.
D

Data for URL 669 saved successfully.
Data for URL 670 saved successfully.
Data for URL 671 saved successfully.
Data for URL 672 saved successfully.
Data for URL 673 saved successfully.
Data for URL 674 saved successfully.
Data for URL 675 saved successfully.
Data for URL 676 saved successfully.
Data for URL 677 saved successfully.
Data for URL 678 saved successfully.
Data for URL 679 saved successfully.
Data for URL 680 saved successfully.
Data for URL 681 saved successfully.
Data for URL 682 saved successfully.
Data for URL 683 saved successfully.
Data for URL 684 saved successfully.
Data for URL 685 saved successfully.
Data for URL 686 saved successfully.
Data for URL 687 saved successfully.
Data for URL 688 saved successfully.
Data for URL 689 saved successfully.
Data for URL 690 saved successfully.
Data for URL 691 saved successfully.
Data for URL 692 saved successfully.
Data for URL 693 saved successfully.
Data for URL 694 saved successfully.
Data for URL 695 saved successfully.
D

Data for URL 891 saved successfully.
Data for URL 892 saved successfully.
Data for URL 893 saved successfully.
Data for URL 894 saved successfully.
Data for URL 895 saved successfully.
Data for URL 896 saved successfully.
Data for URL 897 saved successfully.
Data for URL 898 saved successfully.
Data for URL 899 saved successfully.
Data for URL 900 saved successfully.
Data for URL 901 saved successfully.
Data for URL 902 saved successfully.
Data for URL 903 saved successfully.
Data for URL 904 saved successfully.
Data for URL 905 saved successfully.
Data for URL 906 saved successfully.
Data for URL 907 saved successfully.
Data for URL 908 saved successfully.
Data for URL 909 saved successfully.
Data for URL 910 saved successfully.
Data for URL 911 saved successfully.
Data for URL 912 saved successfully.
Data for URL 913 saved successfully.
Data for URL 914 saved successfully.
Data for URL 915 saved successfully.
Data for URL 916 saved successfully.
Data for URL 917 saved successfully.
D

In [43]:
# Create a directory for storing the metadata text files
folder_name = "metaData"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Iterate over each URL
for index, url in enumerate(urls):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the metadata from the webpage
        metadata = soup.get_text().split("\n")[:20]

        # Generate the output file name based on the index
        output_file = f"{folder_name}/meta{index}.txt"

        # Save the metadata to the output text file
        with open(output_file, "w") as file:
            file.write("\n".join(metadata))

        print(f"Metadata for URL {index} saved successfully.")
    else:
        print(f"Failed to retrieve metadata from URL {index}")


Metadata for URL 0 saved successfully.
Metadata for URL 1 saved successfully.
Metadata for URL 2 saved successfully.
Metadata for URL 3 saved successfully.
Metadata for URL 4 saved successfully.
Metadata for URL 5 saved successfully.
Metadata for URL 6 saved successfully.
Metadata for URL 7 saved successfully.
Metadata for URL 8 saved successfully.
Metadata for URL 9 saved successfully.
Metadata for URL 10 saved successfully.
Metadata for URL 11 saved successfully.
Metadata for URL 12 saved successfully.
Metadata for URL 13 saved successfully.
Metadata for URL 14 saved successfully.
Metadata for URL 15 saved successfully.
Metadata for URL 16 saved successfully.
Metadata for URL 17 saved successfully.
Metadata for URL 18 saved successfully.
Metadata for URL 19 saved successfully.
Metadata for URL 20 saved successfully.
Metadata for URL 21 saved successfully.
Metadata for URL 22 saved successfully.
Metadata for URL 23 saved successfully.
Metadata for URL 24 saved successfully.
Metadata f

Metadata for URL 203 saved successfully.
Metadata for URL 204 saved successfully.
Metadata for URL 205 saved successfully.
Metadata for URL 206 saved successfully.
Metadata for URL 207 saved successfully.
Metadata for URL 208 saved successfully.
Metadata for URL 209 saved successfully.
Metadata for URL 210 saved successfully.
Metadata for URL 211 saved successfully.
Metadata for URL 212 saved successfully.
Metadata for URL 213 saved successfully.
Metadata for URL 214 saved successfully.
Metadata for URL 215 saved successfully.
Metadata for URL 216 saved successfully.
Metadata for URL 217 saved successfully.
Metadata for URL 218 saved successfully.
Metadata for URL 219 saved successfully.
Metadata for URL 220 saved successfully.
Metadata for URL 221 saved successfully.
Metadata for URL 222 saved successfully.
Metadata for URL 223 saved successfully.
Metadata for URL 224 saved successfully.
Metadata for URL 225 saved successfully.
Metadata for URL 226 saved successfully.
Metadata for URL

Metadata for URL 403 saved successfully.
Metadata for URL 404 saved successfully.
Metadata for URL 405 saved successfully.
Metadata for URL 406 saved successfully.
Metadata for URL 407 saved successfully.
Metadata for URL 408 saved successfully.
Metadata for URL 409 saved successfully.
Metadata for URL 410 saved successfully.
Metadata for URL 411 saved successfully.
Metadata for URL 412 saved successfully.
Metadata for URL 413 saved successfully.
Metadata for URL 414 saved successfully.
Metadata for URL 415 saved successfully.
Metadata for URL 416 saved successfully.
Metadata for URL 417 saved successfully.
Metadata for URL 418 saved successfully.
Metadata for URL 419 saved successfully.
Metadata for URL 420 saved successfully.
Metadata for URL 421 saved successfully.
Metadata for URL 422 saved successfully.
Metadata for URL 423 saved successfully.
Metadata for URL 424 saved successfully.
Metadata for URL 425 saved successfully.
Metadata for URL 426 saved successfully.
Metadata for URL

Metadata for URL 603 saved successfully.
Metadata for URL 604 saved successfully.
Metadata for URL 605 saved successfully.
Metadata for URL 606 saved successfully.
Metadata for URL 607 saved successfully.
Metadata for URL 608 saved successfully.
Metadata for URL 609 saved successfully.
Metadata for URL 610 saved successfully.
Metadata for URL 611 saved successfully.
Metadata for URL 612 saved successfully.
Metadata for URL 613 saved successfully.
Metadata for URL 614 saved successfully.
Metadata for URL 615 saved successfully.
Metadata for URL 616 saved successfully.
Metadata for URL 617 saved successfully.
Metadata for URL 618 saved successfully.
Metadata for URL 619 saved successfully.
Metadata for URL 620 saved successfully.
Metadata for URL 621 saved successfully.
Metadata for URL 622 saved successfully.
Metadata for URL 623 saved successfully.
Metadata for URL 624 saved successfully.
Metadata for URL 625 saved successfully.
Metadata for URL 626 saved successfully.
Metadata for URL

Metadata for URL 803 saved successfully.
Metadata for URL 804 saved successfully.
Metadata for URL 805 saved successfully.
Metadata for URL 806 saved successfully.
Metadata for URL 807 saved successfully.
Metadata for URL 808 saved successfully.
Metadata for URL 809 saved successfully.
Metadata for URL 810 saved successfully.
Metadata for URL 811 saved successfully.
Metadata for URL 812 saved successfully.
Metadata for URL 813 saved successfully.
Metadata for URL 814 saved successfully.
Metadata for URL 815 saved successfully.
Metadata for URL 816 saved successfully.
Metadata for URL 817 saved successfully.
Metadata for URL 818 saved successfully.
Metadata for URL 819 saved successfully.
Metadata for URL 820 saved successfully.
Metadata for URL 821 saved successfully.
Metadata for URL 822 saved successfully.
Metadata for URL 823 saved successfully.
Metadata for URL 824 saved successfully.
Metadata for URL 825 saved successfully.
Metadata for URL 826 saved successfully.
Metadata for URL

Metadata for URL 1003 saved successfully.
Metadata for URL 1004 saved successfully.
Metadata for URL 1005 saved successfully.
Metadata for URL 1006 saved successfully.
Metadata for URL 1007 saved successfully.
Metadata for URL 1008 saved successfully.
Metadata for URL 1009 saved successfully.
Metadata for URL 1010 saved successfully.
Metadata for URL 1011 saved successfully.
Metadata for URL 1012 saved successfully.
Metadata for URL 1013 saved successfully.
Metadata for URL 1014 saved successfully.
Metadata for URL 1015 saved successfully.
Metadata for URL 1016 saved successfully.
Metadata for URL 1017 saved successfully.
Metadata for URL 1018 saved successfully.
Metadata for URL 1019 saved successfully.
Metadata for URL 1020 saved successfully.
Metadata for URL 1021 saved successfully.
Metadata for URL 1022 saved successfully.
Metadata for URL 1023 saved successfully.
Metadata for URL 1024 saved successfully.
Metadata for URL 1025 saved successfully.
Metadata for URL 1026 saved succes